In [ ]:
from dataset import GTSequenceDataset
from torch.utils.data import DataLoader, ConcatDataset

SEQ_IN_LEN = 30
SEQ_OUT_LEN = 20
SEQ_TOTAL_LEN = 50
BATCH_SIZE = 1024
STEPS = 4
NOISE_COEFFICIENT = 0
NOISE_PROB = 0
NOISE_COEFFICIENT = 0.15
NOISE_PROB = 0

BASE_DIR = '../../Datasets/'
# train_dataset = ConcatDataset([
#     GTSequenceDataset.from_roots([f'{BASE_DIR}DanceTrack/train'], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB),
#     GTSequenceDataset.from_roots([f'{BASE_DIR}MOT20/train'], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB),
#     GTSequenceDataset.from_roots([f'{BASE_DIR}MOT17/train'], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB),
# ])
# val_dataset = ConcatDataset([
#     GTSequenceDataset.from_roots([f'{BASE_DIR}DanceTrack/val'], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB),
#     GTSequenceDataset.from_roots([f'{BASE_DIR}MOT20/val'], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB),
#     GTSequenceDataset.from_roots([f'{BASE_DIR}MOT17/val'], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB),
# ])
train_dataset = GTSequenceDataset.from_roots([
    f'{BASE_DIR}/SportsMOT/train',
    f'{BASE_DIR}DanceTrack/train',
    f'{BASE_DIR}MOT17/train',
    f'{BASE_DIR}MOT20/train'
], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB)

val_dataset = GTSequenceDataset.from_roots([
    f'{BASE_DIR}/SportsMOT/val',
    f'{BASE_DIR}DanceTrack/val',
    f'{BASE_DIR}MOT17/val',
    f'{BASE_DIR}MOT20/val'
# ], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=0, noise_prob=0)
], seq_in_len=SEQ_IN_LEN, seq_out_len=SEQ_OUT_LEN, seq_total_len=SEQ_TOTAL_LEN, steps=STEPS, noise_coeff=NOISE_COEFFICIENT, noise_prob=NOISE_PROB)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

print(f'Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}')

In [ ]:
from transformer_encoder import MotionTransformer
from loss import LossFunction
from torch import optim
import math, torch, torch.nn as nn
import torch.nn.functional as F
import random
import os


DEVICE = 'cuda'
model = MotionTransformer(
    input_dim=13,
    output_dim=5,
    d_model=256,
    # d_model=512,
    # nhead=16,
    nhead=32,
    num_layers=6,
    dim_ff=512,
    dropout=0.15,
).to(DEVICE)
criterion = LossFunction(loss1_coeff=1, loss2_coeff=0, loss3_coeff=0, loss4_coeff=0)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")


In [ ]:
LR = 1e-5
NUM_EPOCHS = 20

optimizer = optim.AdamW(
    model.parameters(),
    lr=LR,
    betas=(0.9, 0.999),
    weight_decay=1e-4,
    eps=1e-8
)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(
#         optimizer,
#         mode='min',
#         factor=0.5,
#         patience= 11,  # Reduce LR before early stopping
#         verbose=True,
#         min_lr=LR * 0.001
#     )
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS + 1)
best_val_loss = float("inf")
log_file = open('file.log', 'w')
log_file.close()

for epoch in range(1, NUM_EPOCHS + 1):
    train_loss = model.train_one_epoch(train_loader, optimizer, criterion)
    val_loss = model.evaluate(val_loader, criterion)

    scheduler.step()

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        model.save_weight('pretrained/transformer-encoder-d256-ff512-nh32-1l-n3.pth')
        # model.save_weight('pretrained/transformer-encoder-d512-ff512-2l.pth')

    current_lr = scheduler.get_last_lr()[0]
    print(f"Epoch {epoch}: Train Loss = {train_loss:.8f}, Val Loss = {val_loss:.8f}, LR = {current_lr:.8f}")
    log_file = open('file.log', 'a')
    log_file.write(f"Epoch {epoch}: Train Loss = {train_loss:.8f}, Val Loss = {val_loss:.8f}, LR = {current_lr:.8f}\n")
    log_file.close()

print("Training complete. Best Val Loss:", best_val_loss)